In [1]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
import os

def apply_gabor_filter(image, ksize, sigma, theta, lambd, gamma):
    kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, 0)
    return cv2.filter2D(image, -1, kernel)

def extract_features(image):
    # Apply Gabor filters with different orientations
    thetas = [np.pi * i / 4 for i in range(4)]  # 0, 45, 90, 135 degrees
    features = []
    for theta in thetas:
        filtered_image = apply_gabor_filter(image, ksize=7, sigma=1, theta=theta, lambd=7, gamma=0.5)
        features.append(np.mean(filtered_image))
        features.append(np.std(filtered_image))
    return features

# Read images from directory
path = '/home/rtlink/jiwon/paper_ws/heightmaps_30'
image_files = [os.path.join(path, f) for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
images = [cv2.imread(f, cv2.IMREAD_GRAYSCALE) for f in image_files]

# Extract features from images
X = [extract_features(img) for img in images]

# Apply k-means clustering
n_clusters = 5  # You can change this value depending on your needs
kmeans = KMeans(n_clusters=n_clusters).fit(X)

# Create new folders for clusters
base_output_folder = '/home/rtlink/jiwon/paper_ws/분류 기준/clusters'
if not os.path.exists(base_output_folder):
    os.mkdir(base_output_folder)

for i in range(n_clusters):
    cluster_folder = os.path.join(base_output_folder, f"cluster_{i}")
    if not os.path.exists(cluster_folder):
        os.mkdir(cluster_folder)

# Save images to respective clusters
for i, label in enumerate(kmeans.labels_):
    output_path = os.path.join(base_output_folder, f"cluster_{label}", os.path.basename(image_files[i]))
    cv2.imwrite(output_path, images[i])

print(f"Images have been clustered and saved in {base_output_folder}")


Images have been clustered and saved in /home/rtlink/jiwon/paper_ws/clusters
